In [91]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy.cluster.vq as scv
import pickle

# os.system("cd $CAFFE_ROOT")
caffe_root = os.environ["CAFFE_ROOT"]
os.chdir(caffe_root)
print caffe_root
sys.path.insert(0, caffe_root + 'python')
import caffe

caffe.set_mode_gpu()
caffe.set_device(2)
option = 'lenet5'
if option == 'lenet5':
    prototxt = '3_prototxt_solver/lenet5/train_val.prototxt'             
    caffemodel = '4_model_checkpoint/lenet5/lenet5.caffemodel'
    iters = 100
    dir_t = '2_results/kmeans/lenet5/'
elif option == 'alexnet':
    prototxt = '3_prototxt_solver/L2/train_val.prototxt'             
    caffemodel = '4_model_checkpoint/alexnet/alexnet9x.caffemodel'  
    iters = 1000
    dir_t = '2_results/kmeans/alexnet/'
elif option == 'vgg':
    prototxt = '3_prototxt_solver/vgg16/train_val.prototxt'             
    caffemodel = '4_model_checkpoint/vgg16/vgg16_12x.caffemodel'  
    iters = 1000
    dir_t = '2_results/kmeans/vgg16/'

log = dir_t + 'log_accu'

/home/songhan/pruning/


In [92]:
choice = [64,16]
net = caffe.Net(prototxt, caffemodel, caffe.TRAIN)

layers = ["conv1", "conv2", "ip1", "ip2"]
num_c = [8, 8, 8, 8]

# layers = ["ip2"]
# num_c =[4]
print "layers TBD: ", layers
print "num_c = ", num_c

layers TBD:  ['conv1', 'conv2', 'ip1', 'ip2']
num_c =  [8, 8, 8, 8]


In [93]:
print "==============1 Perform K-means============="
codebook = {}
for idx, layer in enumerate(layers):
    print "Eval layer:", layer
    W = net.params[layer][0].data.flatten()
    W = W[np.where(W != 0)]
    std = np.std(W)
    initial_uni = np.linspace(-4 * std, 4 * std, num_c[idx]-1)
    codebook[layer],_= scv.kmeans(W, initial_uni)    
    codebook[layer] = np.append(0.0, codebook[layer])
    print "codebook:", codebook[layer]
    print "codebook size:", len(codebook[layer])

==============1 Perform K-means=============
Eval layer: conv1
codebook: [ 0.         -0.29499978 -0.03177995  0.25980002  0.51812828]
codebook size: 5
Eval layer: conv2
codebook: [ 0.         -0.14003672 -0.08655009 -0.04095863  0.05826917  0.12395576
  0.21972357]
codebook size: 7
Eval layer: ip1
codebook: [ 0.         -0.07194122 -0.04054011 -0.01950156  0.01798702  0.03304647
  0.05246379  0.08222321]
codebook size: 8
Eval layer: ip2
codebook: [ 0.         -0.25988275 -0.17693673 -0.10709237  0.13464746  0.23517904]
codebook size: 6


In [94]:
print "================2 Perform quantization=============="
codeDict={}
maskCode={}
for layer in layers:
    print "Quantize layer:", layer
    W = net.params[layer][0].data
    codes, dist = scv.vq(W.flatten(), codebook[layer])
    W_q = np.reshape(codebook[layer][codes], W.shape)
    net.params[layer][0].data[...] = W_q

    maskCode[layer] = np.reshape(codes, W.shape)
    codeBookSize = len(codebook[layer])    
    print "W_q.shape=", W_q.shape        
    print "codebook length=", codeBookSize
    print "maskcode:", maskCode[layer].flatten().shape
    print "maskcode:", np.flatnonzero(maskCode[layer]).shape
    a = maskCode[layer].flatten()
    b = xrange(len(a))
#     print a
#     print b

    codeDict[layer]={}
    for i in xrange(len(a)):
        codeDict[layer].setdefault(a[i], []).append(b[i])
#     print "codeDict  is",codeDict
#     print maskCode[layer]
    

================2 Perform quantization==============
Quantize layer: conv1
W_q.shape= (20, 1, 5, 5)
codebook length= 5
maskcode: (500,)
maskcode: (280,)
Quantize layer: conv2
W_q.shape= (50, 20, 5, 5)
codebook length= 7
maskcode: (25000,)
maskcode: (2728,)
Quantize layer: ip1
W_q.shape= (500, 800)
codebook length= 8
maskcode: (400000,)
maskcode: (30875,)
Quantize layer: ip2
W_q.shape= (10, 500)
codebook length= 6
maskcode: (5000,)
maskcode: (958,)


In [95]:
print "================3 Perform fintuning=============="
# print codebook
learning_rate=1e-5
decay_rate = 0.99 
momentum=0.9
update='rmsprop'
import time
start_time=time.time()
step_cache={}
for i in xrange(3000):
    net.forward()
    net.backward()
    for layer in layers:
        if not layer in step_cache: 
            step_cache[layer]={}
        diff=net.params[layer][0].diff.flatten()
        W1 =  net.params[layer][0].data
        codeBookSize=len(codebook[layer])
        for code in xrange(codeBookSize):
            if code==0: continue;
            indexes = codeDict[layer][code]
            diff_ave=np.sum(diff[indexes])/len(indexes)

            if update == 'sgd':
                dx = -learning_rate * diff_ave
            elif update == 'momentum':
                if not code in step_cache[layer]:
                    step_cache[layer][code] = 0
                dx = momentum * step_cache[layer][code] - learning_rate * diff_ave
                step_cache[layer][code] = dx                
            elif update == 'rmsprop':
                if not code in step_cache[layer]:
                    step_cache[layer][code] = 0
                step_cache[layer][code] =  decay_rate * step_cache[layer][code] + (1.0 - decay_rate) * diff_ave ** 2
                dx = -(learning_rate * diff_ave) / np.sqrt(step_cache[layer][code] + 1e-8)
            elif update == 'adagrad':
                if not code in step_cache[layer]:
                    step_cache[layer][code] = 0
                step_cache[layer][code] +=  diff_ave ** 2
                dx = -(learning_rate * diff_ave) / np.sqrt(step_cache[layer][code] + 1e-8)
            
            codebook[layer][code] += dx
        W2 = codebook[layer][maskCode[layer]]

#         if lr==0:
#             assert ((W1==W2).all())
        
        net.params[layer][0].data[...]=W2

    if i%200==0:
        print "iteration:", i, "codebook:", codebook["ip2"]
    

print "time elapsed: ", time.time()-start_time 

print "============ Test Accuracy on Training Set ========="
correct = 0
for test_it in range(50000/64):
    net.forward()
    correct += sum(net.blobs['ip2'].data.argmax(1)
                   == net.blobs['label'].data)
print correct / float(50000)

================3 Perform fintuning==============
iteration: 0 codebook: [ 0.         -0.25990896 -0.17694944 -0.10711319  0.13463184  0.23521768]
iteration: 200 codebook: [ 0.         -0.26377599 -0.1814741  -0.11143175  0.13921597  0.23923433]
iteration: 400 codebook: [ 0.         -0.26640841 -0.18419175 -0.11412992  0.1419364   0.24178728]
iteration: 600 codebook: [ 0.         -0.26865944 -0.18660347 -0.11640809  0.14427963  0.24407232]
iteration: 800 codebook: [ 0.         -0.27076142 -0.18888835 -0.11860535  0.14656414  0.2459383 ]
iteration: 1000 codebook: [ 0.         -0.27279905 -0.19110092 -0.12074002  0.14874192  0.2477962 ]
iteration: 1200 codebook: [ 0.         -0.27485651 -0.19320269 -0.12287282  0.150913    0.24944088]
iteration: 1400 codebook: [ 0.         -0.27690424 -0.19528797 -0.12503956  0.15302393  0.25120896]
iteration: 1600 codebook: [ 0.         -0.27888461 -0.19737946 -0.12713386  0.15510681  0.25265595]
iteration: 1800 codebook: [ 0.         -0.28079498 -0.199

original accuracy: 0.9984

lr=1e-5, 3000 iterations:

sgd: 0.99584 / time elapsed:  22.2082271576

momentum: 0.99638 / time elapsed:  22.2547438145

rmsprop: 0.99678 / time elapsed:  30.419727087

adagrad: 0.9952 / time elapsed:  22.2563259602

In [ ]:
print "============ fine tune without codebook on Training Set ========="
print "batch size=",net.blobs['label'].data.shape

import time
start_time = time.time()
for i in xrange(1000):
    net.forward()
    net.backward()
    for layer in layers:        
        diff=net.params[layer][0].diff
        W=    net.params[layer][0].data
        W -= 0.000001*diff    
        net.params[layer][0].data[...]=W
        
print time.time()-start_time
correct = 0
for test_it in range(50000/64):
    net.forward()
    correct += sum(net.blobs['ip2'].data.argmax(1)
                   == net.blobs['label'].data)
print correct / float(50000)